# Загрузка данны

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2, f_classif, SelectKBest
from collections import Counter
import geopy
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import time
# Загрузка данных
file_path = 'D:/coding/project3/hotels.csv'
hotels = pd.read_csv(file_path)
hotels.fillna(0, inplace=True)

# Вывод информации о данных для определения нужных столбцов
print(hotels.info())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

# Очистка данных

In [41]:
# Очистка данных
def fill_missing_coordinates(df, address_column, lat_column, lng_column):
    geolocator = Nominatim(user_agent="hotel_locator")
    cache = {}

    def geocode_address(address):
        if address in cache:
            return cache[address]
        try:
            location = geolocator.geocode(address, timeout=10)
            if location:
                cache[address] = (location.latitude, location.longitude)
                return location.latitude, location.longitude
            else:
                return (np.nan, np.nan)
        except GeocoderTimedOut:
            return (np.nan, np.nan)

    for index, row in tqdm(df[df[lat_column] == 0].iterrows(), total=df[df[lat_column] == 0].shape[0]):
        lat, lng = geocode_address(row[address_column])
        df.at[index, lat_column] = lat
        df.at[index, lng_column] = lng
        # Добавляем задержку для избежания блокировки со стороны API
        time.sleep(1)

    df.dropna(subset=[lat_column, lng_column], inplace=True)
    return df

hotels = fill_missing_coordinates(hotels, 'hotel_address', 'lat', 'lng')

  0%|          | 0/2448 [00:00<?, ?it/s]

 70%|██████▉   | 1709/2448 [34:07<15:19,  1.24s/it] 

3. Исследование данных (качество визуализации, наличие идей, гипотез, комментариев)

In [ ]:
# 1. Сколько уникальных названий отелей представлено в наборе данных?
unique_hotels = hotels['hotel_name'].nunique()
print(f'Уникальных названий отелей: {unique_hotels}')

# 2. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
latest_review_date = hotels['review_date'].max()
print(f'Самый свежий отзыв оставлен: {latest_review_date}')

# 3. Когда был оставлен самый первый отзыв? Введите ответ в формате yyyy-mm-dd.
earliest_review_date = hotels['review_date'].min()
print(f'Самый первый отзыв оставлен: {earliest_review_date}')

# 4. Сколько уникальных тегов представлено в наборе данных?
hotels['tags_list'] = hotels['tags'].apply(lambda x: x.split(','))
unique_tags = set(tag.strip() for tags in hotels['tags_list'] for tag in tags)
print(f'Уникальных тегов: {len(unique_tags)}')

# 5. Какой тег представлен в наибольшем числе отзывов?
tags_counter = Counter(tag.strip() for tags in hotels['tags_list'] for tag in tags)
most_common_tag, most_common_tag_count = tags_counter.most_common(1)[0]
print(f'Самый популярный тег: {most_common_tag} ({most_common_tag_count} отзывов)')

# 6. Из тегов выясните, на сколько ночей чаще всего останавливаются путешественники в отелях.
night_tags = [tag for tag in unique_tags if 'night' in tag.lower()]
night_counter = Counter(tag for tags in hotels['tags_list'] for tag in tags if 'night' in tag.lower())
most_common_night_tag, most_common_night_count = night_counter.most_common(1)[0]
print(f'На сколько ночей чаще всего останавливаются путешественники: {most_common_night_tag} ({most_common_night_count} отзывов)')

# Преобразование столбца review_date в тип данных datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date'])

# Добавление столбцов с годом, месяцем и днем отзыва
hotels['review_year'] = hotels['review_date'].dt.year
hotels['review_month'] = hotels['review_date'].dt.month
hotels['review_day'] = hotels['review_date'].dt.day

# Выводим гистограмму распределения отзывов по годам
plt.figure(figsize=(10, 5))
sns.histplot(hotels['review_year'], kde=False, bins=10)
plt.title('Распределение отзывов по годам')
plt.xlabel('Год')
plt.ylabel('Количество отзывов')
plt.show()

# Выводим гистограмму распределения длины негативных отзывов
hotels['negative_review_length'] = hotels['negative_review'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 5))
sns.histplot(hotels['negative_review_length'], kde=True)
plt.title('Распределение длины негативных отзывов')
plt.xlabel('Длина негативного отзыва (количество слов)')
plt.ylabel('Количество отзывов')
plt.show()

# Выводим гистограмму распределения длины позитивных отзывов
hotels['positive_review_length'] = hotels['positive_review'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 5))
sns.histplot(hotels['positive_review_length'], kde=True)
plt.title('Распределение длины позитивных отзывов')
plt.xlabel('Длина позитивного отзыва (количество слов)')
plt.ylabel('Количество отзывов')
plt.show()

4. Генерация признаков

In [ ]:
from textblob import TextBlob

# Преобразуем категориальные признаки в dummy-переменные
columns_to_drop = ['hotel_address', 'review_date', 'negative_review', 'positive_review', 'tags', 'lat', 'lng']
hotels.drop(columns=columns_to_drop, inplace=True)

# Функция для определения тональности отзыва
def get_sentiment(review):
    analysis = TextBlob(review)
    return analysis.sentiment.polarity

hotels['negative_sentiment'] = hotels['negative_review'].apply(get_sentiment)
hotels['positive_sentiment'] = hotels['positive_review'].apply(get_sentiment)

# Функция для вычисления длины отзыва
def review_length(review):
    return len(review.split())

hotels['negative_review_length'] = hotels['negative_review'].apply(review_length)
hotels['positive_review_length'] = hotels['positive_review'].apply(review_length)

# Преобразуем категориальные признаки в dummy-переменные
categorical_columns = hotels.select_dtypes(include=['object']).columns
for col in categorical_columns:
    hotels = reduce_categories(hotels, col, top_n=10)

hotels = pd.get_dummies(hotels, columns=categorical_columns, drop_first=True)

5. Отбор признаков

In [ ]:
import seaborn as sns

# Корреляционная матрица
corr_matrix = hotels.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f")
plt.title("Корреляционная матрица признаков")
plt.show()

# Выбор значимых признаков
num_cols = ['total_number_of_reviews', 
            'review_total_negative_word_counts', 
            'review_total_positive_word_counts', 
            'days_since_review', 
            'additional_number_of_scoring']

cat_cols = ['average_score', 
            'review_year', 
            'review_month', 
            'review_day']

# Оценка значимости категориальных признаков с помощью теста хи-квадрат
imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')
plt.title('Значимость категориальных признаков')
plt.show()

# Оценка значимости непрерывных признаков с помощью ANOVA
imp_num = pd.Series(f_classif(X[num_cols], y)[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')
plt.title('Значимость непрерывных признаков')
plt.show()

6. Преобразование признаков

In [ ]:
from category_encoders import BinaryEncoder

# Применение бинарного кодирования для категориальных признаков
encoder = BinaryEncoder(cols=categorical_columns)
hotels_encoded = encoder.fit_transform(hotels)

7. Качество решения: результат метрики MAPE

In [ ]:
# Разделение данных на обучающие и тестовые наборы
X = hotels.drop(['reviewer_score'], axis=1)
y = hotels['reviewer_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Создание и обучение модели
regr = RandomForestRegressor(n_estimators=100)
regr.fit(X_train, y_train)

# Предсказание значений на тестовом наборе данных
y_pred = regr.predict(X_test)

# Оценка качества модели
mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
print('MAPE:', mape)

# Отбор лучших признаков
X_new = SelectKBest(score_func=f_classif, k=10).fit_transform(X, y)

# Разделение данных на обучающие и тестовые наборы с новыми признаками
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y, test_size=0.25, random_state=42)

# Создание и обучение новой модели
regr_new = RandomForestRegressor(n_estimators=100)
regr_new.fit(X_train_new, y_train_new)

# Предсказание значений на тестовом наборе данных с новыми признаками
y_pred_new = regr_new.predict(X_test_new)

# Оценка качества новой модели
mape_new = metrics.mean_absolute_percentage_error(y_test_new, y_pred_new)
print('MAPE после отбора признаков:', mape_new)